# LAPD & LA City Aircraft Fleet Analysis

This notebook demonstrates how to use the `hangarbay` Python API to analyze aircraft fleets. We'll look at helicopters and aircraft operated by the Los Angeles Police Department and LA City Fire Department.

## Setup

First, install and import hangarbay:


In [ ]:
# Install (only needed once)
# !pip install hangarbay

import hangarbay as hb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')


## Load FAA Data

This downloads and processes the FAA aircraft registry (~400MB, one-time operation).
Data is stored in `~/.hangarbay/data/` so it's accessible from any notebook.


In [ ]:
# One-time setup - downloads and processes FAA data
hb.load_data()

# Check data status
info = hb.status()
print(f"\nData snapshot: {info['snapshot_date']}")
print(f"Data age: {info['age_days']} days")
print(f"Total aircraft: {info['aircraft_count']:,}")


## LAPD Air Support Division

Let's find all aircraft owned by LAPD. Since the FAA data uses various forms of the name, we'll use wildcard search with the `|` operator for OR logic.


In [ ]:
# Search for LAPD aircraft using wildcard OR logic
lapd = hb.fleet("LAPD|Los Angeles Police")

print(f"Found {len(lapd)} aircraft registered to LAPD\n")
lapd.head()


### Fleet Summary


In [ ]:
# Show owner variations
print("Owner name variations in FAA data:")
print(lapd['owner_name'].unique())

# Fleet statistics
print(f"\nTotal aircraft: {len(lapd)}")
print(f"Valid registrations: {(lapd['reg_status'] == 'Valid').sum()}")
print(f"\nManufacturers:")
print(lapd['maker'].value_counts())


### Aircraft Details


In [ ]:
# Display full fleet with key details
lapd_display = lapd[['n_number', 'maker', 'model', 'year_mfr', 'reg_status']].copy()
lapd_display['n_number'] = 'N' + lapd_display['n_number'].astype(str)
lapd_display.columns = ['N-Number', 'Manufacturer', 'Model', 'Year', 'Status']
lapd_display.sort_values('N-Number')


## LA City Fire Department

Now let's look at fire department aircraft. LA City Fire Department might also be registered under various names.


In [ ]:
# Search for LA City Fire Department aircraft
lafd = hb.fleet("Los Angeles Fire|LAFD", state="CA")

print(f"Found {len(lafd)} aircraft registered to LA Fire Department\n")

if len(lafd) > 0:
    print("Owner name(s):")
    print(lafd['owner_name'].unique())
    
    # Display fleet
    lafd_display = lafd[['n_number', 'maker', 'model', 'year_mfr', 'reg_status']].copy()
    lafd_display['n_number'] = 'N' + lafd_display['n_number'].astype(str)
    lafd_display.columns = ['N-Number', 'Manufacturer', 'Model', 'Year', 'Status']
    display(lafd_display)
else:
    print("Note: LAFD may use contracted/leased aircraft not registered directly to them.")


In [ ]:
# Search for all LA City government aircraft
la_city = hb.fleet("Los Angeles City|City of Los Angeles", state="CA")

print(f"Found {len(la_city)} aircraft registered to LA City entities\n")

if len(la_city) > 0:
    # Show unique owners
    print("LA City Government Entities:")
    for owner in sorted(la_city['owner_name'].unique()):
        count = (la_city['owner_name'] == owner).sum()
        print(f"  {owner}: {count} aircraft")
    
    # Display all aircraft
    print(f"\nComplete Fleet:")
    city_display = la_city[['n_number', 'owner_name', 'maker', 'model', 'year_mfr']].copy()
    city_display['n_number'] = 'N' + city_display['n_number'].astype(str)
    city_display.columns = ['N-Number', 'Owner', 'Manufacturer', 'Model', 'Year']
    display(city_display.sort_values('Owner'))


## Individual Aircraft Lookup

For detailed information on a specific aircraft, use the `search()` function:


In [ ]:
# Look up a specific LAPD helicopter
if len(lapd) > 0:
    # Get the first N-number from our LAPD fleet
    example_n = lapd.iloc[0]['n_number']
    
    print(f"Detailed lookup for N{example_n}:\n")
    aircraft = hb.search(example_n)
    
    # Display key fields
    display_fields = ['n_number', 'maker', 'model', 'year_mfr', 'reg_status', 
                      'owner_name', 'city', 'state', 'reg_expiration']
    for col in display_fields:
        if col in aircraft.columns:
            value = aircraft.iloc[0][col]
            if pd.notna(value) and value != '':
                print(f"{col}: {value}")


## Export Data

Save the LAPD fleet data for further analysis:


In [ ]:
# Export to CSV
if len(lapd) > 0:
    output_file = 'lapd_fleet.csv'
    lapd.to_csv(output_file, index=False)
    print(f"✓ Exported {len(lapd)} aircraft to {output_file}")


## Summary

This notebook demonstrated:

- ✅ Installing and loading FAA data with `hangarbay`
- ✅ Using wildcard search to find fleets across name variations
- ✅ Analyzing fleet composition and manufacturers
- ✅ Searching for multiple city agencies (Police, Fire, etc.)
- ✅ Looking up individual aircraft details
- ✅ Exporting data for further work

The `hangarbay` API makes it easy to go from "I wonder how many helicopters LAPD has" to actionable insights in minutes, not hours.
